## Load Food Dataset

In [15]:
# Shamelessly stolen from groupmates
import pandas as pd
pd.set_option('display.max_colwidth', None)
import json


# Data Directory: 
# upking_images_directory = '/shared/data/upking/'
upking_annotation_file_path = '/shared/data/upking/cat_data.json'


# Method 1: Using pandas.read_json directly
df = pd.read_json(upking_annotation_file_path)

df.head()


,id,file_name,url,captions
0,228915,COCO_train2014_000000228915.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000228915.jpg,"[A lady wearing a ring and two bracelets holds a gray cat., An uncomfortable looking cat wearing a tight white fabric collar, Someone holding a grey cat that has been shaved. , A dark gray cat with shaved fur is held by a woman., a cat with a collar sitting by a woman]"
1,280866,COCO_train2014_000000280866.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000280866.jpg,"[A person crossing their legs while a cat walks against a wall., a person wearing red tennis shoes and a black cat, Person with red tennis shoes with feet up near black cat., A person's feet in red shoes next to a black cat., A black cat walking a ledge in a dark room.]"
2,467333,COCO_train2014_000000467333.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000467333.jpg,"[A fluffy cat laying on the arm of a couch., A black and white photo of a cat with a remote control., A cat is laying on the couch arm by a remote control., A cat is sitting on the arm of a chair with a remote., A cat sitting on a couch arm next to a remote control.]"
3,476960,COCO_train2014_000000476960.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000476960.jpg,"[A cat standing with a banana in its mouth., There is a cat that is trying to eat a banana, a house cat playing with a ripe banana, A cat with a banana in its mouth., The cat has the banana in his mouth.]"
4,118960,COCO_train2014_000000118960.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000118960.jpg,"[A couple of men standing next to each other near a parking lot., Two men standing near a parked car having a conversation., Some guys standing on a side walk by a car., Two people standing on the sidewalk, one with a backpack that has a cat inside it., Two men have a conversation on a sidewalk.]"


In [16]:
df.shape

(20, 4)

## Generate Multiple Choice Question

In [17]:
# Generate random choice in [A, B, C, D]
import random

def generate_random_choice():
    return random.choice(['A', 'B', 'C', 'D'])

In [18]:
df['multiple_choice_solution'] = df.apply(lambda x: generate_random_choice(), axis=1)

In [19]:
df.tail()

,id,file_name,url,captions,multiple_choice_solution
15,260978,COCO_train2014_000000260978.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000260978.jpg,"[A cat is rolling around on a couch, An orange cat lying on its side on a bed., A cat laying on a couch with a sheet and a remote on it., An orange cat on a sofa with a remote, A cat upside on the couch near a remote]",C
16,142601,COCO_train2014_000000142601.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000142601.jpg,"[One black cat and one orange cat sleeping on a bed near pillows., Two cats curled up on opposite sides of a bed., two cats one black the other a light brown sleeping on a bed, A couple of cats are laying on a large bed., Cats sleeping on a quilted bed in a room.]",A
17,436014,COCO_train2014_000000436014.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000436014.jpg,"[A man using a laptop with cats sitting next to him on the table., Cats sit on a desk while a young man uses a laptop., A man on a laptop while two cats sit with him., A man uses his laptop while cats sit around his table., A man works on a computer white cats sit on the table.]",D
18,103447,COCO_train2014_000000103447.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000103447.jpg,"[A cat standing in a open suitcase on some cloths., A cat that is resting in a full bag of luggage., a cat lays inside of a suitcase with clothes in it , A cat standing in a laundry hamper looking down., An animal sitting in a suit case filled with clothing.]",B
19,372759,COCO_train2014_000000372759.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000372759.jpg,"[a black cat is hiding in a box with shoes, A cat and some shoes side by side., There is a cat sitting next to a pair of boots., A cat sits among a pile of shoes in a confined space., A black cat peers at a bunch of black shoes.]",A


In [ ]:
# Generate multiple choice questions

In [20]:
import os
from openai import OpenAI
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  
)


def generate_multiple_choice_question(reference_caption, correct_choice): 
    # Define the prompt to generate inferior choices
    prompt = f"""
    The ground truth caption is:
    "{reference_caption}"

    Generate three inferior captions that include either inaccurate details, or are non-fluent with syntactic errors. 
    Format the result as a multiple-choice question. 
    Question title should be "Which of the following captions best describes the painting?".
    The correct choice should be placed at choice "{correct_choice}". 
    Ensure the incorrect choices are realistic but clearly wrong.
    Do not generate special symbols such as '*'.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.7,
        max_tokens=200,
    )

    # Extract the generated multiple-choice question
    question = response.choices[0].message.content    
    
    return question

In [21]:
df.apply(lambda x: print(x['captions'][0], x['multiple_choice_solution']), axis=1)

A lady wearing a ring and two bracelets holds a gray cat. A
A person crossing their legs while a cat walks against a wall. C
A fluffy cat laying on the arm of a couch. D
A cat standing with a banana in its mouth. C
A couple of men standing next to each other near a parking lot. D
A multi-colored cat laying on a rug next to some bottles and a glass door. B
A cat sitting in a sink over a cup of something. C
A cat sits on a wooden chair with a purse. C
A cat is hiding under a blanket and peering out. A
A cat is laying on top of a laptop computer. A
A dog and a cat stand side by side D
Someone standing in a room with their shoes on their feet.  D
there is a black tuxedo cat looking in the mirror B
Two cats play with each other near some computers. B
One cat plays with a banana while another lies down on a rug. C
A cat is rolling around on a couch C
One black cat and one orange cat sleeping on a bed near pillows. A
A man using a laptop with cats sitting next to him on the table. D
A cat sta

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
dtype: object

In [22]:
df['multiple_choice_question'] = df.apply(lambda x: generate_multiple_choice_question(x['captions'][0], x['multiple_choice_solution']), axis=1)

In [23]:
df.head()

,id,file_name,url,captions,multiple_choice_solution,multiple_choice_question
0,228915,COCO_train2014_000000228915.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000228915.jpg,"[A lady wearing a ring and two bracelets holds a gray cat., An uncomfortable looking cat wearing a tight white fabric collar, Someone holding a grey cat that has been shaved. , A dark gray cat with shaved fur is held by a woman., a cat with a collar sitting by a woman]",A,**Which of the following captions best describes the painting?**\n\nA) A lady wearing a ring and two bracelets holds a gray cat.\n\nB) A man with a watch and three rings holding a small dog.\n\nC) A woman wearing a necklace and a hat holds a brown cat.\n\nD) A lady holding gray cat with no jewelry on hands.
1,280866,COCO_train2014_000000280866.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000280866.jpg,"[A person crossing their legs while a cat walks against a wall., a person wearing red tennis shoes and a black cat, Person with red tennis shoes with feet up near black cat., A person's feet in red shoes next to a black cat., A black cat walking a ledge in a dark room.]",C,Which of the following captions best describes the painting?\n\nA) A dog runs across the field while a person jumps over a fence.\n\nB) Two cats are playing in a sunny meadow with no people around.\n\nC) A person crossing their legs while a cat walks against a wall.\n\nD) A bird flies over a pond with ducks swimming nearby.
2,467333,COCO_train2014_000000467333.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000467333.jpg,"[A fluffy cat laying on the arm of a couch., A black and white photo of a cat with a remote control., A cat is laying on the couch arm by a remote control., A cat is sitting on the arm of a chair with a remote., A cat sitting on a couch arm next to a remote control.]",D,Which of the following captions best describes the painting?\n\nA) A tiny dog sitting on a chair in the garden.\n\nB) A fluffy cat sleeping under the table.\n\nC) A fluffy cat laying on the sofa's pillow.\n\nD) A fluffy cat laying on the arm of a couch.
3,476960,COCO_train2014_000000476960.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000476960.jpg,"[A cat standing with a banana in its mouth., There is a cat that is trying to eat a banana, a house cat playing with a ripe banana, A cat with a banana in its mouth., The cat has the banana in his mouth.]",C,Which of the following captions best describes the painting?\n\nA) A cat sitting with an apple in its paw.\n\nB) A cat stand with bananas on its head.\n\nC) A cat standing with a banana in its mouth.\n\nD) A dog standing with a banana in its mouth.
4,118960,COCO_train2014_000000118960.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000118960.jpg,"[A couple of men standing next to each other near a parking lot., Two men standing near a parked car having a conversation., Some guys standing on a side walk by a car., Two people standing on the sidewalk, one with a backpack that has a cat inside it., Two men have a conversation on a sidewalk.]",D,Which of the following captions best describes the painting?\n\nA) Two men are sitting on a bench in a crowded park.\n\nB) A group of women walking together through a busy street.\n\nC) A single man looking at cars in a remote parking lot.\n\nD) A couple of men standing next to each other near a parking lot.


In [24]:
# Save the annotation with multiple choice question to output file

# Convert DataFrame to a list of dictionaries
list_of_dicts = df.to_dict(orient="records")

# Save the list of dictionaries to a JSON file
output_file = "/shared/data/upking/upking_annotation_with_MCQ.json"
with open(output_file, "w") as file:
    json.dump(list_of_dicts, file, indent=4)

print(f"DataFrame saved as a list of dictionaries in {output_file}")

DataFrame saved as a list of dictionaries in /shared/data/upking/upking_annotation_with_MCQ.json


## Perform Multiple Choice Selection

In [1]:
# Load annotation with multiple choice question data file
import pandas as pd
pd.set_option('display.max_colwidth', None)
import json


# Data Directory: 
#food_images_directory = '/shared/data/food_data/food_images/'
upking_MCQ_file_path = '/shared/data/upking/upking_annotation_with_MCQ.json'


# Method 1: Using pandas.read_json directly
df = pd.read_json(upking_MCQ_file_path)

df.head()

,id,file_name,url,captions,multiple_choice_solution,multiple_choice_question
0,228915,COCO_train2014_000000228915.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000228915.jpg,"[A lady wearing a ring and two bracelets holds a gray cat., An uncomfortable looking cat wearing a tight white fabric collar, Someone holding a grey cat that has been shaved. , A dark gray cat with shaved fur is held by a woman., a cat with a collar sitting by a woman]",A,**Which of the following captions best describes the painting?**\n\nA) A lady wearing a ring and two bracelets holds a gray cat.\n\nB) A man with a watch and three rings holding a small dog.\n\nC) A woman wearing a necklace and a hat holds a brown cat.\n\nD) A lady holding gray cat with no jewelry on hands.
1,280866,COCO_train2014_000000280866.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000280866.jpg,"[A person crossing their legs while a cat walks against a wall., a person wearing red tennis shoes and a black cat, Person with red tennis shoes with feet up near black cat., A person's feet in red shoes next to a black cat., A black cat walking a ledge in a dark room.]",C,Which of the following captions best describes the painting?\n\nA) A dog runs across the field while a person jumps over a fence.\n\nB) Two cats are playing in a sunny meadow with no people around.\n\nC) A person crossing their legs while a cat walks against a wall.\n\nD) A bird flies over a pond with ducks swimming nearby.
2,467333,COCO_train2014_000000467333.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000467333.jpg,"[A fluffy cat laying on the arm of a couch., A black and white photo of a cat with a remote control., A cat is laying on the couch arm by a remote control., A cat is sitting on the arm of a chair with a remote., A cat sitting on a couch arm next to a remote control.]",D,Which of the following captions best describes the painting?\n\nA) A tiny dog sitting on a chair in the garden.\n\nB) A fluffy cat sleeping under the table.\n\nC) A fluffy cat laying on the sofa's pillow.\n\nD) A fluffy cat laying on the arm of a couch.
3,476960,COCO_train2014_000000476960.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000476960.jpg,"[A cat standing with a banana in its mouth., There is a cat that is trying to eat a banana, a house cat playing with a ripe banana, A cat with a banana in its mouth., The cat has the banana in his mouth.]",C,Which of the following captions best describes the painting?\n\nA) A cat sitting with an apple in its paw.\n\nB) A cat stand with bananas on its head.\n\nC) A cat standing with a banana in its mouth.\n\nD) A dog standing with a banana in its mouth.
4,118960,COCO_train2014_000000118960.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000118960.jpg,"[A couple of men standing next to each other near a parking lot., Two men standing near a parked car having a conversation., Some guys standing on a side walk by a car., Two people standing on the sidewalk, one with a backpack that has a cat inside it., Two men have a conversation on a sidewalk.]",D,Which of the following captions best describes the painting?\n\nA) Two men are sitting on a bench in a crowded park.\n\nB) A group of women walking together through a busy street.\n\nC) A single man looking at cars in a remote parking lot.\n\nD) A couple of men standing next to each other near a parking lot.


### Use llava model

In [2]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests 

In [3]:
# Load model from local directory 
model_path = '/shared/model/llava-v1.6-mistral-7b-hf'

processor = LlavaNextProcessor.from_pretrained(model_path)

model = LlavaNextForConditionalGeneration.from_pretrained(model_path, torch_dtype=torch.float16, low_cpu_mem_usage=True, load_in_4bit=True) 
#model.to("cuda:0")

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
import skimage.io as io

In [5]:
# helper function: 
def perform_multiple_choice_task_llava(img_url, question):
    image = io.imread(img_url)
    
    conversation = [
        {

          "role": "user",
          "content": [
              {"type": "text", "text": question + "\nOnly return the correct choice with a single letter."},
              {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda:0")

    # autoregressively complete prompt
    output = model.generate(**inputs, max_new_tokens=150)
    output = processor.decode(output[0], skip_special_tokens=True)

    # mcq_question = output.split('[/INST]')[0].split('[INST] ')[1].strip()
    mcq_answer = output.split('[/INST]')[1].strip()
    # del image, inputs, output
    # torch.cuda.empty_cache()

    return mcq_answer

In [41]:
def process_in_batches(df, batch_size):
    predictions = []  # 用于存储结果
    for start in range(0, len(df), batch_size):
        # 取出当前批次的数据
        batch = df.iloc[start:start + batch_size]
        batch_predictions = []
        
        for _, row in batch.iterrows():
            # 对每一行执行多选任务
            prediction = perform_multiple_choice_task_llava(row['url'], row['multiple_choice_question'])
            batch_predictions.append(prediction)
        
        # 将当前批次的结果添加到最终列表中
        predictions.extend(batch_predictions)
    
    return predictions




In [42]:
# 设置批量大小（根据显存容量调整）
batch_size = 1

# 执行批量处理，并将预测结果添加为新的列
df['multiple_choice_prediction'] = process_in_batches(df, batch_size)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 14.57 GiB of which 6.75 MiB is free. Process 1034175 has 9.60 GiB memory in use. Including non-PyTorch memory, this process has 4.96 GiB memory in use. Of the allocated memory 4.79 GiB is allocated by PyTorch, and 33.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
df['multiple_choice_prediction'] = df.apply(lambda x: perform_multiple_choice_task_llava(x['url'], x['multiple_choice_question']), axis=1)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Expanding inputs for image tokens in LLaVa-NeXT should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:Non

In [7]:
df.head(1)

,id,file_name,url,captions,multiple_choice_solution,multiple_choice_question,multiple_choice_prediction
0,228915,COCO_train2014_000000228915.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000228915.jpg,"[A lady wearing a ring and two bracelets holds a gray cat., An uncomfortable looking cat wearing a tight white fabric collar, Someone holding a grey cat that has been shaved. , A dark gray cat with shaved fur is held by a woman., a cat with a collar sitting by a woman]",A,**Which of the following captions best describes the painting?**\n\nA) A lady wearing a ring and two bracelets holds a gray cat.\n\nB) A man with a watch and three rings holding a small dog.\n\nC) A woman wearing a necklace and a hat holds a brown cat.\n\nD) A lady holding gray cat with no jewelry on hands.,A


In [8]:
df[['multiple_choice_solution', 'multiple_choice_prediction']]

,multiple_choice_solution,multiple_choice_prediction
0,A,A
1,C,C
2,D,C
3,C,C
4,D,D
5,B,B
6,C,C
7,C,C
8,A,A
9,A,A


In [10]:
# Save the MCQ result

# Convert DataFrame to a list of dictionaries
list_of_dicts = df.to_dict(orient="records")

# Save the list of dictionaries to a JSON file
output_file = "/shared/data/upking/upking_annotation_with_MCQ_result.json"

with open(output_file, "w") as file:
    json.dump(list_of_dicts, file, indent=4)

print(f"DataFrame saved as a list of dictionaries in {output_file}")

DataFrame saved as a list of dictionaries in /shared/data/upking/upking_annotation_with_MCQ_result.json


## Perform Image Captioning Task

In [15]:
# Load annotation with multiple choice question data file
import pandas as pd
pd.set_option('display.max_colwidth', None)
import json


# Data Directory: 
#food_images_directory = '/shared/data/food_data/food_images/'
food_annotation_file_path = '/shared/data/upking/upking_annotation_with_MCQ.json'


# Method 1: Using pandas.read_json directly
df = pd.read_json(food_annotation_file_path)

df.head(1)

,id,file_name,url,captions,multiple_choice_solution,multiple_choice_question
0,228915,COCO_train2014_000000228915.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000228915.jpg,"[A lady wearing a ring and two bracelets holds a gray cat., An uncomfortable looking cat wearing a tight white fabric collar, Someone holding a grey cat that has been shaved. , A dark gray cat with shaved fur is held by a woman., a cat with a collar sitting by a woman]",A,**Which of the following captions best describes the painting?**\n\nA) A lady wearing a ring and two bracelets holds a gray cat.\n\nB) A man with a watch and three rings holding a small dog.\n\nC) A woman wearing a necklace and a hat holds a brown cat.\n\nD) A lady holding gray cat with no jewelry on hands.


In [19]:
# helper function: 
def perform_image_captioning_task_llava(img_url):
    image = io.imread(img_url)
    
    conversation = [
        {

          "role": "user",
          "content": [
              {"type": "text", "text": "Generate a caption for this image."},
              {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda:0")

    # autoregressively complete prompt
    output = model.generate(**inputs, max_new_tokens=150)
    output = processor.decode(output[0], skip_special_tokens=True)

    caption = output.split('[/INST]')[1].strip()
    print(caption)
    return caption

In [20]:
df['predicted_caption'] = df.apply(lambda x: perform_image_captioning_task_llava(x['url']), axis=1)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"A moment of affection between a human and their feline friend."


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"A cozy evening with a feline friend."


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

"A cozy workspace with feline companions."


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"A curious feline exploring the depths of a suitcase."


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"A curious feline explores a collection of shoes and boots."


In [22]:
df[['captions', 'predicted_caption']]

,captions,predicted_caption
0,"[A lady wearing a ring and two bracelets holds a gray cat., An uncomfortable looking cat wearing a tight white fabric collar, Someone holding a grey cat that has been shaved. , A dark gray cat with shaved fur is held by a woman., a cat with a collar sitting by a woman]","""A moment of affection between a human and their feline friend."""
1,"[A person crossing their legs while a cat walks against a wall., a person wearing red tennis shoes and a black cat, Person with red tennis shoes with feet up near black cat., A person's feet in red shoes next to a black cat., A black cat walking a ledge in a dark room.]","""A cozy evening with a feline friend."""
2,"[A fluffy cat laying on the arm of a couch., A black and white photo of a cat with a remote control., A cat is laying on the couch arm by a remote control., A cat is sitting on the arm of a chair with a remote., A cat sitting on a couch arm next to a remote control.]","""A cozy moment captured in black and white."""
3,"[A cat standing with a banana in its mouth., There is a cat that is trying to eat a banana, a house cat playing with a ripe banana, A cat with a banana in its mouth., The cat has the banana in his mouth.]","""A curious feline explores the world of human snacks."""
4,"[A couple of men standing next to each other near a parking lot., Two men standing near a parked car having a conversation., Some guys standing on a side walk by a car., Two people standing on the sidewalk, one with a backpack that has a cat inside it., Two men have a conversation on a sidewalk.]","""Friends sharing a moment on the sidewalk."""
5,"[A multi-colored cat laying on a rug next to some bottles and a glass door., A cat sitting on the floor looking up, The cat looks upward beside some glass bottles., A very cute cat on a rug by some bottles., Multi colored cat laying on the floor next to door and liquor bottles]","""A curious feline gazes out the window, surrounded by the warmth of a cozy home."""
6,"[A cat sitting in a sink over a cup of something., A tortoiseshell cat stands in a stainless steel sink and drinks from a cup., A cat looking up while sipping from a cup in a sink., A very cute cat sitting in a sink drinking from a cup., furry cat drinking from a paper cup inside a kitchen sink]","""A cozy cat enjoying a warm beverage."""
7,"[A cat sits on a wooden chair with a purse., A cat sitting on a wooden chair next to table., A cat sitting on a chair in a restaurant, A cat sits on a cafe chair with a handbag, Grey and white cat sitting on top of a wooden chair. ]","""A curious feline observing the world from its perch."""
8,"[A cat is hiding under a blanket and peering out., A black cat hiding under a blue blanket., A cat looks out from under a blue blanket., There is a cat lying under a blanket, a cat looking out from underneath a blanket.]","""A cozy hideout for a curious feline."""
9,"[A cat is laying on top of a laptop computer., A cat taking a nap on a laptop., A ginger cat resting on a PC laptop, A brown cat napping on a laptop kept on a sofa., a cat sleeps on a laptop computer on a couch]","""A cozy afternoon nap with a laptop as a pillow."""


In [24]:
# Save the image_captioning result

# Convert DataFrame to a list of dictionaries
list_of_dicts = df.to_dict(orient="records")

# Save the list of dictionaries to a JSON file
output_file = "/shared/data/upking/upking_annotation_with_image_captioning_result.json"

with open(output_file, "w") as file:
    json.dump(list_of_dicts, file, indent=4)

print(f"DataFrame saved as a list of dictionaries in {output_file}")

DataFrame saved as a list of dictionaries in /shared/data/upking/upking_annotation_with_image_captioning_result.json


## Evaluate results

In [25]:
# import libraries
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice

### Multiple Choice Accuracy

In [29]:
# Load annotation with multiple choice question result data file
import pandas as pd
pd.set_option('display.max_colwidth', None)
import json


# Data Directory: 
food_annotation_file_path = '/shared/data/upking/upking_annotation_with_MCQ_result.json'


# Method 1: Using pandas.read_json directly
df = pd.read_json(food_annotation_file_path)

df.head(1)

,id,file_name,url,captions,multiple_choice_solution,multiple_choice_question,multiple_choice_prediction
0,228915,COCO_train2014_000000228915.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000228915.jpg,"[A lady wearing a ring and two bracelets holds a gray cat., An uncomfortable looking cat wearing a tight white fabric collar, Someone holding a grey cat that has been shaved. , A dark gray cat with shaved fur is held by a woman., a cat with a collar sitting by a woman]",A,**Which of the following captions best describes the painting?**\n\nA) A lady wearing a ring and two bracelets holds a gray cat.\n\nB) A man with a watch and three rings holding a small dog.\n\nC) A woman wearing a necklace and a hat holds a brown cat.\n\nD) A lady holding gray cat with no jewelry on hands.,A


In [30]:
def calculate_multiple_choice_question_accuracy(df):
    # Calculate accuracy
    accuracy = (df["multiple_choice_solution"] == df["multiple_choice_prediction"]).mean()

    print(f"Prediction Accuracy: {accuracy * 100:.2f}%") 
    return accuracy

In [31]:
calculate_multiple_choice_question_accuracy(df)

Prediction Accuracy: 85.00%


0.85

### Caption Quality


In [35]:
# Load annotation with multiple choice question result data file
import pandas as pd
pd.set_option('display.max_colwidth', None)
import json


# Data Directory: 
food_annotation_file_path = '/shared/data/upking/upking_annotation_with_image_captioning_result.json'


# Method 1: Using pandas.read_json directly
df = pd.read_json(food_annotation_file_path)

df.head(1)

,id,file_name,url,captions,multiple_choice_solution,multiple_choice_question,predicted_caption
0,228915,COCO_train2014_000000228915.jpg,http://images.cocodataset.org/train2014/COCO_train2014_000000228915.jpg,"[A lady wearing a ring and two bracelets holds a gray cat., An uncomfortable looking cat wearing a tight white fabric collar, Someone holding a grey cat that has been shaved. , A dark gray cat with shaved fur is held by a woman., a cat with a collar sitting by a woman]",A,**Which of the following captions best describes the painting?**\n\nA) A lady wearing a ring and two bracelets holds a gray cat.\n\nB) A man with a watch and three rings holding a small dog.\n\nC) A woman wearing a necklace and a hat holds a brown cat.\n\nD) A lady holding gray cat with no jewelry on hands.,"""A moment of affection between a human and their feline friend."""


In [ ]:
# BLEU

In [36]:
from nltk.translate.bleu_score import sentence_bleu


# BLEU Evaluation (Average across multiple references)
def evaluate_bleu(df):
    bleu_scores = []
    
    for _, row in df.iterrows():
        references = row['captions']  # List of reference captions
        candidate = row['predicted_caption']  # Predicted caption

        # Tokenize the candidate and reference captions
        tokenized_references = [ref.strip('"').split() for ref in references]  # List of tokenized references
        tokenized_candidate = candidate.strip('"').split()  # Tokenized candidate

        # Compute BLEU for all references
        row_bleu_scores = [
            sentence_bleu([ref], tokenized_candidate) for ref in tokenized_references
        ]
        
        # Average across references
        bleu_scores.append(sum(row_bleu_scores) / len(row_bleu_scores))
    
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"Average BLEU: {avg_bleu:.4f}")
    return bleu_scores

In [37]:
evaluate_bleu(df)

Average BLEU: 0.0000


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

[1.0579048440249822e-231,
 7.268137743732178e-232,
 8.705390904782547e-156,
 6.311011137147783e-232,
 4.2637485317189243e-156,
 8.691680632243206e-232,
 7.140215571639016e-232,
 7.751011273422461e-232,
 9.930306306361165e-232,
 3.7889263068295175e-155,
 4.0784964889236096e-232,
 9.126231777561599e-232,
 2.2439443022320825e-155,
 1.1119137413006488e-231,
 3.324938679824386e-155,
 1.614663279194461e-155,
 1.1518583712221898e-155,
 3.801257184539974e-232,
 9.752516669670534e-156,
 1.1214958877252064e-155]

In [ ]:
# METEOR

In [38]:
import nltk
nltk.download('wordnet')
from nltk.translate.meteor_score import meteor_score

# METEOR Evaluation (Average across multiple references)
def evaluate_meteor(df):
    meteor_scores = []
    
    for _, row in df.iterrows():
        references = row['captions']  # List of reference captions
        candidate = row['predicted_caption'].strip('"')  # Predicted caption (raw string)

        # Compute METEOR for all references
        row_meteor_scores = [
            meteor_score([[ref]], [candidate]) for ref in references
        ]
        
        # Average across references
        meteor_scores.append(sum(row_meteor_scores) / len(row_meteor_scores))
    
    avg_meteor = sum(meteor_scores) / len(meteor_scores)
    print(f"Average METEOR: {avg_meteor:.4f}")
    return meteor_scores


[nltk_data] Downloading package wordnet to /home/upking/nltk_data...


In [39]:
evaluate_meteor(df)

Average METEOR: 0.0000


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
# ROUGE

In [40]:
# ROUGE Evaluation (Average across multiple references)
def evaluate_rouge(df):
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    for _, row in df.iterrows():
        references = row['captions']  # List of reference captions
        candidate = row['predicted_caption']  # Predicted caption
        
        # Compute ROUGE scores for all references
        row_rouge1_scores, row_rouge2_scores, row_rougeL_scores = [], [], []
        for ref in references:
            scores = scorer.score(ref, candidate.strip('"'))
            row_rouge1_scores.append(scores['rouge1'].fmeasure)
            row_rouge2_scores.append(scores['rouge2'].fmeasure)
            row_rougeL_scores.append(scores['rougeL'].fmeasure)
        
        # Average across references
        rouge1_scores.append(sum(row_rouge1_scores) / len(row_rouge1_scores))
        rouge2_scores.append(sum(row_rouge2_scores) / len(row_rouge2_scores))
        rougeL_scores.append(sum(row_rougeL_scores) / len(row_rougeL_scores))
    
    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)
    
    print(f"Average ROUGE-1: {avg_rouge1:.4f}")
    print(f"Average ROUGE-2: {avg_rouge2:.4f}")
    print(f"Average ROUGE-L: {avg_rougeL:.4f}")
    return rouge1_scores, rouge2_scores, rougeL_scores

In [41]:
evaluate_rouge(df)

Average ROUGE-1: 0.2365
Average ROUGE-2: 0.0283
Average ROUGE-L: 0.2213


([0.16600414078674947,
  0.20055039559683524,
  0.16126984126984129,
  0.11281045751633986,
  0.24015345268542196,
  0.14026607894209578,
  0.3167919799498746,
  0.10538011695906432,
  0.25705882352941173,
  0.41301587301587295,
  0.09180426556587548,
  0.13356521739130436,
  0.3705597326649958,
  0.3145454545454545,
  0.3806878306878307,
  0.2590056022408963,
  0.2875057208237986,
  0.1550326797385621,
  0.2686549707602339,
  0.35558441558441556],
 [0.0,
  0.0,
  0.04444444444444444,
  0.0,
  0.038095238095238085,
  0.0,
  0.0,
  0.0,
  0.0,
  0.09049707602339181,
  0.0,
  0.0,
  0.0891640866873065,
  0.07858585858585859,
  0.05174825174825175,
  0.049624060150375945,
  0.06031746031746031,
  0.0,
  0.044444444444444446,
  0.019999999999999997],
 [0.16600414078674947,
  0.20055039559683524,
  0.16126984126984129,
  0.11281045751633986,
  0.19923273657289,
  0.12426607894209578,
  0.2945697577276524,
  0.10538011695906432,
  0.25705882352941173,
  0.3907936507936508,
  0.09180426556587

In [ ]:
# CIDEr and SPICE (unchanged, since they handle multiple references internally)

In [42]:
def evaluate_cider(df):
    ref_dict = {str(idx): row['captions'] for idx, row in df.iterrows()}
    cand_dict = {str(idx): [row['predicted_caption']] for idx, row in df.iterrows()}
    
    cider_scorer = Cider()
    score, _ = cider_scorer.compute_score(ref_dict, cand_dict)
    print(f"Average CIDEr: {score:.4f}")
    return score

In [43]:
evaluate_cider(df)

Average CIDEr: 0.0591


0.05907432453946156

In [44]:
def evaluate_spice(df):
    ref_dict = {str(idx): row['captions'] for idx, row in df.iterrows()}
    cand_dict = {str(idx): [row['predicted_caption']] for idx, row in df.iterrows()}
    
    spice_scorer = Spice()
    score, _ = spice_scorer.compute_score(ref_dict, cand_dict)
    print(f"Average SPICE: {score:.4f}")
    return score